In [1]:
!pip install -U requests==2.32.5 --quiet
!pip install -U "langgraph>=0.2.26"  --quiet
!pip install langchain langchain-community sentence-transformers faiss-cpu transformers accelerate torch --quiet
!python -m pip show langgraph


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.2/471.2 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.3/208.3 kB 16.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langch

In [10]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "NotesRAGApp"
os.environ["LANGCHAIN_API_KEY"] = "<<API_KEY>>"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"


In [11]:
from langsmith import Client

client = Client()
print("Connected to LangSmith ✅, current project:", os.environ.get("LANGCHAIN_PROJECT"))


Connected to LangSmith ✅, current project: NotesRAGApp


In [12]:
from typing import TypedDict
from sentence_transformers import SentenceTransformer
from transformers import pipeline
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langgraph.graph import StateGraph
from langgraph.checkpoint.memory import MemorySaver
from datetime import datetime
import pandas as pd

notes_db = [
    {"id": 1, "tags": ["AI", "LangGraph"], "text": "LangGraph lets you build agent workflows as graphs with memory and control flow."},
    {"id": 2, "tags": ["AI"], "text": "Retrieval-Augmented Generation (RAG) connects LLMs to external knowledge to reduce hallucinations."},
    {"id": 3, "tags": ["Python"], "text": "FastAPI is a fast, modern framework for building web APIs in Python."},
    {"id": 4, "tags": ["AI", "Embeddings"], "text": "Sentence-Transformers provide high-quality text embeddings for semantic search."}
]




In [13]:
# Initialize embedding model wrapper for LangChain
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

texts = [n["text"] for n in notes_db]
metas = [{"tags": n["tags"], "id": n["id"]} for n in notes_db]

# Build FAISS vector store (LangChain handles embeddings internally)
vectorstore = FAISS.from_texts(texts, embedding=embedding_model, metadatas=metas)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

In [14]:
from langgraph.graph import StateGraph
from typing import TypedDict

class RAGState(TypedDict):
    query: str
    context: str
    answer: str

builder = StateGraph(state_schema=RAGState)

def retriever_node(state):
    docs = retriever.invoke(state["query"])
    state["context"] = "\n".join([d.page_content for d in docs])
    return state

def summarizer_node(state):
    prompt = f"Answer concisely using only this context:\n{state['context']}\n\nQuestion: {state['query']}\nAnswer:"
    response = llm.invoke(prompt)
    state["answer"] = response
    return state

builder.add_node("retriever", retriever_node)
builder.add_node("summarizer", summarizer_node)
builder.add_edge("retriever", "summarizer")
builder.set_entry_point("retriever")
builder.set_finish_point("summarizer")

checkpointer = MemorySaver()

app = builder.compile(checkpointer=checkpointer)


In [15]:
from datetime import datetime
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import json

semantic_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

metrics = []

def run_query(query, session_id="default"):
  """Run a quey through RAG graph with session memory + metrics tracking"""
  config = {"configurable":{"thread_id": session_id}}
  state = app.invoke({"query": query, "context": "", "answer": ""}, config=config)

  # Print human-readable response
  print(f"\nSession: {session_id}")
  print(f"Query: {query}")
  print(f"Answer: {state['answer'][:250]}\n")

  # completeness = round(min(1.0, len(state["context"].split()) / (len(query.split()) * 5)), 2)
  # faithfulness = 1.0 if any(w in state["answer"].lower() for w in query.lower().split()[:2]) else 0.8
  # mismatch = 1-faithfulness

  query_emb = semantic_model.encode(query, convert_to_tensor=True)
  answer_emb = semantic_model.encode(state["answer"], convert_to_tensor=True)
  context_emb = semantic_model.encode(state["context"], convert_to_tensor=True)

  # Faithfulness → how close the answer is to the retrieved context
  faithfulness = float(util.cos_sim(answer_emb, context_emb))
  # Retrieval relevance → how close the retrieved context is to the user’s query
  retrieval_relevance = float(util.cos_sim(query_emb, context_emb))
  # Completeness → estimated by context length ratio
  completeness = round(min(1.0, len(state["context"].split()) / (len(query.split()) * 5)), 2)

  print(f"Completeness: {completeness}")
  print(f"Faithfulness: {faithfulness}")
  print(f"Retrieval Relevence: {retrieval_relevance}")

  metrics.append({
      "session": session_id,
      "query": query,
      "completeness": completeness,
      "faithfulness": faithfulness,
      "Retrieval Relevence": retrieval_relevance,
      "timestamp": datetime.now().isoformat()
  })

  print(json.dumps(metrics))

  return state


In [16]:
from langchain_community.llms import HuggingFacePipeline

flan = pipeline("text2text-generation", model="google/flan-t5-base", device_map="auto", max_new_tokens=150)
llm = HuggingFacePipeline(pipeline=flan)


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu
/tmp/ipython-input-1083948495.py:4: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=flan)


In [17]:
run_query("What is LangGraph?", "user001")
run_query("How does RAG reduce hallucinations?", "user001")
run_query("Explain FastAPI briefly", "user001")



Session: user001
Query: What is LangGraph?
Answer: LangGraph lets you build agent workflows as graphs with memory and control flow. Sentence-Transformers provide high-quality text embeddings for semantic search

Completeness: 1.0
Faithfulness: 0.996911883354187
Retrieval Relevence: 0.4405069947242737
[{"session": "user001", "query": "What is LangGraph?", "completeness": 1.0, "faithfulness": 0.996911883354187, "Retrieval Relevence": 0.4405069947242737, "timestamp": "2025-11-08T17:14:44.546735"}]

Session: user001
Query: How does RAG reduce hallucinations?
Answer: connects LLMs to external knowledge

Completeness: 0.96
Faithfulness: 0.445770263671875
Retrieval Relevence: 0.44643956422805786
[{"session": "user001", "query": "What is LangGraph?", "completeness": 1.0, "faithfulness": 0.996911883354187, "Retrieval Relevence": 0.4405069947242737, "timestamp": "2025-11-08T17:14:44.546735"}, {"session": "user001", "query": "How does RAG reduce hallucinations?", "completeness": 0.96, "faithfuln

{'query': 'Explain FastAPI briefly',
 'context': 'FastAPI is a fast, modern framework for building web APIs in Python.\nLangGraph lets you build agent workflows as graphs with memory and control flow.',
 'answer': 'FastAPI is a fast, modern framework for building web APIs in Python. LangGraph lets you build agent workflows as graphs with memory and control flow.'}